In [4]:
from mysql.connector import *

class customer:
    def init(self):
        pass
    def make_new_customer(self,name,email,password,phone_number):
        make_customer_instance=connect(host='localhost',user='root',password='',database='movieticketsystem')
        make_customer_cursor=make_customer_instance.cursor()
        makecustomer="insert into user(user_type, name, email, password, phone_number) values(%s,%s,%s,%s,%s)"
        str=("customer",name,email,password,phone_number)
        make_customer_cursor.execute(makecustomer,str)
        print("customer make succesully")
        make_customer_instance.commit()

    def ticket_booking(self, user_id):
        # Establish database connection
        ticket_booking_instance = connect(host='localhost', user='root', password='', database='movieticketsystem')
        ticket_booking_cursor = ticket_booking_instance.cursor()

        # Fetch movies available
        ticket_booking_cursor.execute("SELECT movie_id, movie_name FROM movie")
        movies = ticket_booking_cursor.fetchall`()

        # Display available movies and prompt for selection
        print("Available Movies:")
        for movie_id, movie_name in movies:
            print(movie_id, ": ", movie_name)
    
        # Validate user input for movie selection
        selected_movie = None
        while selected_movie is None:
            try:
                selected_movie = int(input("Enter the movie id from the above list: "))
                if selected_movie not in [movie[0] for movie in movies]:
                    print("Please enter a valid movie id.")
                    selected_movie = None
            except ValueError:
                print("Please enter a valid integer value for movie id.")

        # Fetch booking details for selected movie
        ticket_booking_cursor.execute("SELECT * FROM bookings WHERE movie_id = %s", (selected_movie,))
        bookings = ticket_booking_cursor.fetchall()

        # Display available bookings for the selected movie
        print("Available Bookings:")
        for book_id, theatre_id, movie_id, shows, prices in bookings:
            print(f"{book_id}: Theater ID: {theatre_id}, Movie ID: {movie_id}, Shows: {shows}, Prices: {prices}")

        # Prompt for selection of booking id
        selected_book_id = None
        while selected_book_id is None:
            try:
                selected_book_id = int(input("Enter the booking id from the above list: "))
                if selected_book_id not in [booking[0] for booking in bookings]:
                    print("Please enter a valid booking id.")
                    selected_book_id = None
            except ValueError:
                print("Please enter a valid integer value for booking id.")

        # Fetch available show times for the selected booking
        ticket_booking_cursor.execute("SELECT shows FROM bookings WHERE book_id = %s", (selected_book_id,))
        shows = ticket_booking_cursor.fetchone()[0].split(',')

        # Display available show times
        print("Available Shows:")
        for show in shows:
            print(show)

        # Prompt for selection of show time
        selected_show = None
        while selected_show is None:
            selected_show = input("Enter the show time from the above list: ")
            if selected_show not in shows:
                print("Please enter a valid show time.")

        # Fetch occupied seats for the user
        ticket_booking_cursor.execute("SELECT seats FROM ticket_booking WHERE user_id = %s", (user_id,))
        occupied_seats_row = ticket_booking_cursor.fetchone()
        occupied_seats = occupied_seats_row[0].split(',') if occupied_seats_row is not None else []

        print("The occupied seats are: ", occupied_seats)

        # Prompt for selection of seats
        selected_seats = []
        while True:
            select_seat = input("Enter the number of seat or enter -1 to stop the seat selection: ")
            if select_seat == "-1":
                break
            elif select_seat in occupied_seats:
                print("This seat is already booked.")
            else:
                print("Seat booked successfully")
                selected_seats.append(select_seat)

        # Calculate total price of tickets
        ticket_booking_cursor.execute("SELECT prices FROM bookings WHERE book_id = %s", (selected_book_id,))
        price_per_ticket = ticket_booking_cursor.fetchone()[0]
        total_price = price_per_ticket * len(selected_seats)
        print("Total price of tickets = ", total_price)

        # Insert booking details into database
        booking_str = "INSERT INTO ticket_booking (user_id, theatre_id, seats, show_time) VALUES (%s, %s, %s, %s)"
        booking_data = (user_id, selected_book_id, ','.join(selected_seats), selected_show)
        ticket_booking_cursor.execute(booking_str, booking_data)
        ticket_booking_instance.commit()
        print("Your ticket has been booked successfully.")


    def booking_cancel(self, user_id):
        booking_cancel_instance = connect(host='localhost', user='root', password='', database='movieticketsystem')
        booking_cancel_cursor = booking_cancel_instance.cursor()
        data = "SELECT * FROM ticket_booking WHERE user_id=%s"
        booking_cancel_cursor.execute(data, (user_id,))
        result = booking_cancel_cursor.fetchall()
        if result:
            for row in result:
                tk_id, theatre_id, seats, show_time, date, *extra = row
                customer.display(tk_id, theatre_id, seats, show_time, date)
            Cancel_id = int(input("Please Enter Ticket Id For Cancel Ticket From Given List:"))
            cancelticket = "DELETE FROM ticket_booking WHERE tk_id=%s"
            booking_cancel_cursor.execute(cancelticket, (Cancel_id,))
            booking_cancel_instance.commit()
            if booking_cancel_cursor.rowcount > 0:
                print("Your Ticket Canceled Successfully")
        else:
            print("No booking found for the user.")



    def movies_review(self, movie_id):
        movies_review_instance = connect(host='localhost',user='root',password='',database='movieticketsystem')
        movies_review_cursor = movies_review_instance.cursor()
        review = input("Review For Movie:")
        setreview ="insert into review(movie_id,review) values(%s,%s)"
        movies_review_cursor.execute(setreview,(movie_id,review))
        movies_review_instance.commit()
        if movies_review_cursor.rowcount > 0:
            print("Thank You For Your Review")

        
    def display(tk_id,theatre_id,seats,show_time,date):
        print("Ticket ID:",tk_id," Theatre ID:",theatre_id," Seats:",seats," Show Time:",show_time," Date:",date)

class admin:
    def init(self):
        pass

    def validate_admin(self, name, password):
        validate_admin_instance = connect(host='localhost', user='root', password='', database='movieticketsystem')
        validate_admin_cursor = validate_admin_instance.cursor()

        # Check if the name and password match in the admin table
        validate_admin_query = "SELECT COUNT(*) FROM user WHERE name = %s AND password = %s AND user_type = 'admin'"
        validate_admin_cursor.execute(validate_admin_query, (name, password))
        admin_exists = validate_admin_cursor.fetchone()[0]

        if admin_exists > 0:
            print("Admin is valid.")
            return True
        else:
            print("Invalid admin credentials.")
            return False


    def set_movies(self):
        movie_name = str(input("Enter the name of the movie"))
        movie_length = str(input("Enter the length of the movie (in minutes)"))
        while True:
            if any(char.isalpha() for char in movie_length):
                print("Please enter the movie length in integers")
                movie_length = str(input("Enter the length of the movie (in minutes)"))
            else:
                break
        checked_movie_length = int(movie_length)
        movie_category = str(input("Enter the category for the movie: "))
        movie_description = str(input("Enter movie description: "))
        movie_set_instance = connect(host='localhost', user='root', password='', database='movieticketsystem')
        movie_set_cursor = movie_set_instance.cursor()
        make_movie = "insert into movie(movie_name, movie_length, movie_category, movie_description) values (%s,%s,%s,%s)"
        movie_data = (movie_name, checked_movie_length, movie_category, movie_description)
        movie_set_cursor.execute(make_movie, movie_data)
        movie_set_instance.commit()


    def set_screens(self):
        time_choices = {
            1: "9:00", 2: "10:00", 3: "11:00", 4: "12:00", 5: "13:00", 6: "14:00",
            7: "15:00", 8: "16:00", 9: "17:00", 10: "18:00", 11: "19:00", 12: "20:00",
            13: "21:00", 14: "22:00"
        }

        set_screen_instance = connect(host='localhost', user='root', password='', database='movieticketsystem')
        set_screen_cursor = set_screen_instance.cursor()

        # Select movie
        set_screen_cursor.execute("SELECT movie_id, movie_name FROM movie")
        movies = set_screen_cursor.fetchall()
        print("Select a movie from the below list")
        for movie in movies:
            print(movie[0], ".", movie[1])

        movie_choice_id = 0
        while True:
            movie_choice_id = int(input("Enter the movie id from the above list: "))
            if any(movie_choice_id == movie[0] for movie in movies):
                break
            else:
                print("Enter a valid movie id from the above list")

        # Select theatre
        set_screen_cursor.execute("SELECT theatre_id, theatre_name FROM theatre")
        theatres = set_screen_cursor.fetchall()
        print("Select theatre from the below list")
        for theatre in theatres:
            print(theatre[0], ".", theatre[1])

        theatre_choice = 0
        while True:
            theatre_choice = int(input("Enter theatre choice: "))
            if any(theatre_choice == theatre[0] for theatre in theatres):
                break
            else:
                print("Enter a valid theatre id from the above list")

        # Select show timings
        print("Select show timings from the below list")
        for k, v in time_choices.items():
            print(k, v)

        selected_timings = []
        while True:
            timechoice = int(input("Enter id for time or -1 to finish: "))
            if timechoice == -1:
                break
            elif timechoice in time_choices:
                selected_timings.append(time_choices[timechoice])
            else:
                print("Invalid time choice")

        ticket_price = int(input("Enter ticket price: "))

            # Insert data into bookings table
        str_insert = "INSERT INTO bookings (theatre_id, movie_id, shows, prices) VALUES (%s, %s, %s, %s)"
        data = (theatre_choice, movie_choice_id, ','.join(selected_timings), ticket_price)
        set_screen_cursor.execute(str_insert, data)
        set_screen_instance.commit()

        print("Screens set successfully")




    def add_theatre(self):
        set_theatre_instance = connect(host='localhost', user='root', password='', database='movieticketsystem')
        set_theatre_cursor = set_theatre_instance.cursor()
        theatre_name = str(input("Enter theatre name: "))  # Renamed the variable from str to theatre_name_str
        theatre_address = str(input("Enter theatre address"))
        theatre_seats = int(input("Enter max theater seats"))
        str_insert = "INSERT INTO theatre (theatre_name, address, seats) VALUES (%s,%s,%s)"
        data = (theatre_name, theatre_address, theatre_seats)
        set_theatre_cursor.execute(str_insert, data)
        set_theatre_instance.commit()

    def del_theatre(self):
        del_theatre_instance=connect(host='localhost',user='root',password='',database='movieticketsystem')
        del_theatre_cursor=del_theatre_instance.cursor()
        del_theatre_cursor.execute("select theatre_id,theatre_name from theatre")
        result=del_theatre_cursor.fetchall()
        theatre_id=0
        resultchoice=""
        print("select theatre id to delete it")
        for rec in result:
            print(rec[0],": ", rec[1])
            resultchoice+=str(rec[0])
        while True:
            theatre_id=int(input("Enter the id for deletion: "))
            if str(theatre_id) in resultchoice:
                break
            else:
                print("enter valid theatre id")

def name_validations(name):
    if any(char.isdigit() for char in name):
        return False
    else:
        return True
    
def phone_validations(phone):
    if len(phone)==10:
        if phone.isnumeric():
            return True
        return False
    else:
        print("Please enter the number of 10 digits")
        return False
    
def email_validations(email):
    mail_check_instance = connect(host='localhost', user='root', password='', database='movieticketsystem')
    mail_cursor = mail_check_instance.cursor()
    mail_cursor.execute("SELECT email FROM user")
    result = [row[0] for row in mail_cursor.fetchall()]
    if email in result:
        print("The email already exists.")
        return False
    elif "@" in email:
        if email[0].isnumeric():
            return False
        else:
            return True
    else:
        print("Please enter a proper email")
        return False
    
def password_validations(password):
    if len(password) == 8:
        return True
    else:
        return False


# print("Hello there!! Welcome to movie ticketing system")
# print("Enter admin if you want to use admin properties, else press any key and continue")
# the_user=str(input("Enter here: "))

# if (the_user.lower())=="admin":
#     admin_controls=admin()
#     print("Are you an existing admin or wanted to be new one")
#     while True:
#         ad_choice=str(input("Enter 1.for existing admin \t2.for new admin"))
#         if ad_choice=='1':

#             break
#         elif ad_choice=='2':
#             break
#         else:
#             print("Please enter a valid choice")            
# else:
#     pass
def main():
    print("Welcome to the Movie Ticketing System!")
    print("Choose the user type:")
    user_type = input("Enter 'admin' or 'customer': ").lower()

    choice = input("Enter your choice: ")

    if user_type == "admin":
        admin_instance = admin()
        name = input("Enter your name: ")
        password = input("Enter your password: ")

            # Validate admin credentials
        if admin_instance.validate_admin(name, password):
            # Display admin functions
            print("Admin functions:")
            print("1. Set Movies")
            print("2. Set Screens")
            print("3. Add Theatre")
            print("4. Delete Theatre")
            print("5. Exit")
        
            while True:
                admin_choice = input("Enter your admin function choice: ")
                if admin_choice == "1":
                    admin_instance.set_movies()
                elif admin_choice == "2":
                    admin_instance.set_screens()
                elif admin_choice == "3":
                    admin_instance.add_theatre()
                elif admin_choice == "4":
                    admin_instance.del_theatre()
                elif admin_choice == "5":
                    print("Exiting...")
                    break
                else:
                    print("Invalid admin function choice. Please try again.")

        elif choice == "6":
            print("Exiting...")
        else:
            print("Invalid choice. Please try again.")


    elif user_type == 'customer':
        customer_instance = customer()
        while True:
            print("\nCustomer Menu:")
            print("1. Register as a new customer")
            print("2. Book tickets")
            print("3. Cancel booking")
            print("4. Leave movie review")
            print("5. Exit")

            choice = input("Enter your choice: ")

            if choice == "1":
                name = input("Enter your name: ")
                email = input("Enter your email: ")
                while not email_validations(email):
                    email = input("Enter a valid email: ")
                password = input("Enter your password (8 characters): ")
                while not password_validations(password):
                    password = input("Enter a valid password (8 characters): ")
                phone_number = input("Enter your phone number: ")
                while not phone_validations(phone_number):
                    phone_number = input("Enter a valid phone number: ")

                customer_instance.make_new_customer(name, email, password, phone_number)

            elif choice == "2":
                user_id = input("Enter your user ID: ")  # Assuming the user ID is known or retrieved from login
                customer_instance.ticket_booking(user_id)

            elif choice == "3":
                user_id = input("Enter your user ID: ")  # Assuming the user ID is known or retrieved from login
                customer_instance.booking_cancel(user_id)

            elif choice == "4":
                movie_id = input("Enter the movie ID: ")  # Assuming the movie ID is known
                customer_instance.movies_review(movie_id)

            elif choice == "5":
                print("Exiting...")
                break

            else:
                print("Invalid choice. Please try again.")

    else:
        print("Invalid user type. Please enter 'admin' or 'customer'.")

m=main()

Welcome to the Movie Ticketing System!
Choose the user type:


Enter 'admin' or 'customer':  customer
Enter your choice:  1



Customer Menu:
1. Register as a new customer
2. Book tickets
3. Cancel booking
4. Leave movie review
5. Exit


Enter your choice:  1
Enter your name:  aaa
Enter your email:  aaa@gmail.com


DatabaseError: 2003 (HY000): Can't connect to MySQL server on 'localhost:3306' (10061)